In [4]:
import os
import sys
sys.path.append('..')
import pandas as pd

from scripts.extraccion_datos import *

# Crear la carpeta 'data' si no existe
directorio_salida = 'data'
os.makedirs(directorio_salida, exist_ok=True)

# Descargar el archivo Excel
url = 'https://www.ine.es/jaxiT3/files/t/es/xlsx/60272.xlsx?nocab=1'
ruta_salida = os.path.join(directorio_salida, 'datos_ine.xlsx')
descargar_archivo_excel(url, ruta_salida)

# Verificar que la descarga fue exitosa
if os.path.exists(ruta_salida):
    print("Descarga exitosa del archivo Excel.")
else:
    print("Error en la descarga del archivo Excel.")

# Cargar el archivo Excel
datos_excel = cargar_archivo_excel(ruta_salida)

# Leer la hoja de datos
nombre_hoja = datos_excel.sheet_names[0]
df = leer_hoja_excel(datos_excel, nombre_hoja)

# Guardar el DataFrame original con codificación UTF-8
ruta_salida_original = os.path.join(directorio_salida, 'datos_ine_original.xlsx')
df.to_excel(ruta_salida_original, index=False)

# Mostrar las primeras filas y columnas relevantes del DataFrame
print("Primeras filas del DataFrame original:")
print(df.iloc[:5, :10])  # Mostrar las primeras 5 filas y 10 columnas

Descarga exitosa del archivo Excel.


/usr/local/python/3.12.1/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Primeras filas del DataFrame original:
0       Índice de Producción Industrial. (Base 2021)     NaN  NaN  NaN  NaN  \
0                                                NaN     NaN  NaN  NaN  NaN   
1  General y por destino económico de los bienes....     NaN  NaN  NaN  NaN   
2                        Unidades:   Índice,   Tasas     NaN  NaN  NaN  NaN   
3                                                NaN     NaN  NaN  NaN  NaN   
4                                                     Índice  NaN  NaN  NaN   

0  NaN  NaN  NaN  NaN  NaN  
0  NaN  NaN  NaN  NaN  NaN  
1  NaN  NaN  NaN  NaN  NaN  
2  NaN  NaN  NaN  NaN  NaN  
3  NaN  NaN  NaN  NaN  NaN  
4  NaN  NaN  NaN  NaN  NaN  


In [5]:
# Limpieza de datos
print("Iniciando la limpieza de datos...") 

from scripts.limpieza_datos import limpiar_dataframe, reemplazar_nan_por_cero, limpiar_datos_incorrectos

# Limpiar el DataFrame
df_limpio = limpiar_dataframe(df)
archivo_limpio = os.path.join(directorio_salida, 'datos_ine_limpio.xlsx')
df_limpio.to_excel(archivo_limpio, index=False)
print(f"Datos limpiados y guardados en: {archivo_limpio}")

# Reemplazar NaN por 0
df_sin_nan = reemplazar_nan_por_cero(df_limpio)
archivo_sin_nan = os.path.join(directorio_salida, 'datos_ine_sin_nan.xlsx')
df_sin_nan.to_excel(archivo_sin_nan, index=False)
print(f"Datos con NaN reemplazados por 0 y guardados en: {archivo_sin_nan}")

# Limpiar datos incorrectos
df_corregido = limpiar_datos_incorrectos(df_sin_nan)
archivo_corregido = os.path.join(directorio_salida, 'datos_ine_corregido.xlsx')
df_corregido.to_excel(archivo_corregido, index=False)
print(f"Datos incorrectos limpiados y guardados en: {archivo_corregido}")

Iniciando la limpieza de datos...
Datos limpiados y guardados en: data/datos_ine_limpio.xlsx
Datos con NaN reemplazados por 0 y guardados en: data/datos_ine_sin_nan.xlsx
Datos incorrectos limpiados y guardados en: data/datos_ine_corregido.xlsx


In [8]:
# Función para transponer el DataFrame
def transponer_dataframe(df):
    return df.transpose()

# Transponer el DataFrame
df_transpuesto = transponer_dataframe(df_corregido)
archivo_transpuesto = os.path.join(directorio_salida, 'datos_ine_transpuesto.xlsx')
df_transpuesto.to_excel(archivo_transpuesto, index=False)
print(f"Datos transpuestos y guardados en: {archivo_transpuesto}")

# Extraer colecciones de datos
colecciones_datos = [df_transpuesto.iloc[i].tolist() for i in range(len(df_transpuesto))]
print("Colecciones de datos extraídas correctamente.")

Datos transpuestos y guardados en: data/datos_ine_transpuesto.xlsx
Colecciones de datos extraídas correctamente.


In [9]:
import os

# Importar la función guardar_datos_concatenados desde el archivo correspondiente
from scripts.operaciones import guardar_datos_concatenados

# Asegurarse de que el directorio de salida existe
directorio_salida = 'data'
if not os.path.exists(directorio_salida):
    os.makedirs(directorio_salida)

# Guardar las colecciones de datos en un archivo de texto con codificación UTF-8
archivo_lista = os.path.join(directorio_salida, 'lista.txt')
guardar_datos_concatenados(colecciones_datos, archivo_lista)
print(f"Colecciones de datos guardadas en: {archivo_lista}")

Colecciones de datos guardadas en: data/lista.txt


In [48]:
import os
import pandas as pd

# Asegurarse de que el directorio de salida existe
directorio_salida = 'data'
if not os.path.exists(directorio_salida):
    os.makedirs(directorio_salida)

# Cargar el DataFrame transpuesto desde el archivo Excel
ruta_archivo_transpuesto = os.path.join(directorio_salida, 'datos_ine_transpuesto.xlsx')
df_transpuesto = pd.read_excel(ruta_archivo_transpuesto)

# Eliminar las columnas 0 a 13 y 149 a 156
cols_to_drop = list(df_transpuesto.columns[0:12]) + list(df_transpuesto.columns[149:156])
df_transpuesto.drop(cols_to_drop, axis=1, inplace=True)

# Eliminar las celdas donde haya 0
df_transpuesto.replace(0, pd.NA, inplace=True)
df_transpuesto.dropna(how='any', inplace=True)

# Guardar el DataFrame limpio en un nuevo archivo Excel
ruta_archivo_limpio = os.path.join(directorio_salida, 'datos_ine_transpuesto_limpio.xlsx')
df_transpuesto.to_excel(ruta_archivo_limpio, index=False)
print(f"DataFrame limpio guardado en: {ruta_archivo_limpio}")

# Definir las columnas de interés
columnas_interes = [
    'Total industria', 'Bienes de consumo', 'Bienes de consumo duradero',
    'Bienes de consumo no duradero', 'Bienes de equipo', 'Bienes intermedios', 'Energía'
]

# Filtrar el DataFrame para incluir solo las filas de interés
df_filtrado = df_transpuesto[df_transpuesto.iloc[:, 0].isin(columnas_interes)]

# Establecer la primera columna como índice
df_filtrado.set_index(df_filtrado.columns[0], inplace=True)

# Convertir los datos a numéricos, reemplazando no numéricos por NaN
df_filtrado = df_filtrado.apply(pd.to_numeric, errors='coerce')

# Calcular la suma total para cada categoría
sumas_totales = df_filtrado.sum(axis=1)

# Crear un nuevo DataFrame con las sumas totales
df_sumas = pd.DataFrame(sumas_totales, columns=['Total']).transpose()

# Guardar el DataFrame con las sumas en un nuevo archivo Excel
ruta_archivo_sumas = os.path.join(directorio_salida, 'datos_ine_sumas.xlsx')
df_sumas.to_excel(ruta_archivo_sumas, index=True)
print(f"DataFrame con las sumas guardado en: {ruta_archivo_sumas}")

# Imprimir el DataFrame con las sumas
print("Contenido del DataFrame con las sumas:")
print(df_sumas)

DataFrame limpio guardado en: data/datos_ine_transpuesto_limpio.xlsx
DataFrame con las sumas guardado en: data/datos_ine_sumas.xlsx
Contenido del DataFrame con las sumas:
Empty DataFrame
Columns: [Total]
Index: []


In [44]:
import pandas as pd

# Cargar el archivo Excel
file_path = 'data/datos_ine_transpuesto_limpio.xlsx'
df = pd.read_excel(file_path)

# Seleccionar las columnas de interés
columnas_interes = [
    'Total industria', 'Bienes de consumo', 'Bienes de consumo duradero',
    'Bienes de consumo no duradero', 'Bienes de equipo', 'Bienes intermedios', 'Energía'
]
df_filtrado = df[df.iloc[:, 0].isin(columnas_interes)]

# Asegurarse de que los datos sean numéricos y eliminar filas no válidas
df_filtrado = df_filtrado.apply(pd.to_numeric, errors='coerce').dropna()

# Calcular media, varianza y moda
media = df_filtrado.mean().mean()
varianza = df_filtrado.var().mean()
moda = df_filtrado.mode().iloc[0].mean() if not df_filtrado.mode().empty else None

# Imprimir resultados
print(f"MEDIA: {media}")
print(f"VARIANZA: {varianza}")
print(f"MODA: {moda}")


MEDIA: nan
VARIANZA: nan
MODA: None


In [ ]:
class DataRow:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
    
    def __str__(self):
        return ', '.join(f"{key}: {value}" for key, value in self.__dict__.items())
    
    def modificar_valor(self, atributo, valor):
        if atributo in self.__dict__:
            self.__dict__[atributo] = valor
    
    def __eq__(self, other):
        return self.__dict__.get('columna2', None) == other.__dict__.get('columna2', None)
    
    def __lt__(self, other):
        return self.__dict__.get('columna2', None) < other.__dict__.get('columna2', None)
    
    def __add__(self, other):
        return {key: self.__dict__[key] + other.__dict__[key] for key in self.__dict__}
    
    def __sub__(self, other):
        return {key: self.__dict__[key] - other.__dict__[key] for key in self.__dict__}

# Crear objetos de la clase para al menos cinco filas
filas = [DataRow(**df_final.iloc[i].to_dict()) for i in range(5)]

# Probar métodos
for fila in filas:
    print(fila)

# Modificar valores
filas[0].modificar_valor('columna1', 999)
print(f"Fila modificada: {filas[0]}")

# Comparar objetos
print(f"Comparación: {filas[0] == filas[1]}")
print(f"Suma: {filas[0] + filas[1]}")
print(f"Resta: {filas[0] - filas[1]}")